## 크롤링한 랭커들의 레벨 계산

### 유저들이 푼 문제들의 시도횟수, 정답비율 확인

In [1]:
import pandas as pd
import numpy as np

In [2]:
rankers = pd.read_csv("raw_data/rankers.csv")
ranker_problems = pd.read_csv("raw_data/ranker_problems.csv")

recommend_problems = pd.read_csv("final_data/CSV/recommend_problem_infos.csv")

In [3]:
rankers.info()
rankers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34900 entries, 0 to 34899
Data columns (total 4 columns):
id             34900 non-null int64
user           34899 non-null object
correct_cnt    34900 non-null int64
level          34900 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.1+ MB


,id,user,correct_cnt,level
0,0,koosaga,8842,30
1,1,dotorya,7486,27
2,2,xiaowuc1,6246,30
3,3,cki86201,5900,30
4,4,portableangel,5771,24


In [4]:
# null값 가지는 행 찾기
rankers[pd.isnull(rankers).any(axis=1)]

,id,user,correct_cnt,level
32336,32336,NaN,61,999


In [5]:
rankers.fillna("Unknown", inplace = True)

In [6]:
rankers.info()
rankers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34900 entries, 0 to 34899
Data columns (total 4 columns):
id             34900 non-null int64
user           34900 non-null object
correct_cnt    34900 non-null int64
level          34900 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.1+ MB


,id,user,correct_cnt,level
0,0,koosaga,8842,30
1,1,dotorya,7486,27
2,2,xiaowuc1,6246,30
3,3,cki86201,5900,30
4,4,portableangel,5771,24


In [7]:
ranker_problems = ranker_problems.rename({'problem_id' : 'number'}, axis = 1)

In [8]:
ranker_problems.info()
ranker_problems.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6322246 entries, 0 to 6322245
Data columns (total 5 columns):
id         int64
user_id    int64
user       object
number     int64
TF         object
dtypes: int64(3), object(2)
memory usage: 241.2+ MB


,id,user_id,user,number,TF
0,0,0,koosaga,1000,o
1,1,0,koosaga,1001,o
2,2,0,koosaga,1002,o
3,3,0,koosaga,1003,o
4,4,0,koosaga,1004,o


In [9]:
# null값 가지는 행 찾기
ranker_problems[pd.isnull(ranker_problems).any(axis=1)]

,id,user_id,user,number,TF
6159616,6159616,32336,NaN,1000,o
6159617,6159617,32336,NaN,1001,o
6159618,6159618,32336,NaN,1008,o
6159619,6159619,32336,NaN,1085,o
6159620,6159620,32336,NaN,1100,o
...,...,...,...,...,...
6159680,6159680,32336,NaN,2675,x
6159681,6159681,32336,NaN,3034,x
6159682,6159682,32336,NaN,5337,x
6159683,6159683,32336,NaN,10170,x


In [10]:
# null 값 가지는 열 찾기
ranker_problems.columns[ranker_problems.isna().any()].tolist()

['user']

In [11]:
ranker_problems.fillna("Unknown", inplace = True)

In [12]:
recommend_problems.info()
recommend_problems.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9267 entries, 0 to 9266
Data columns (total 13 columns):
seq               9267 non-null int64
number            9267 non-null int64
name              9267 non-null object
algorithm         9267 non-null object
algorithm_id      9267 non-null object
information       9267 non-null object
correct_user      9267 non-null int64
submission_cnt    9267 non-null int64
correct_rate      9267 non-null float64
level             9267 non-null int64
avg_try           9267 non-null float64
time_limit        9267 non-null object
memory_limit      9267 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 941.3+ KB


,seq,number,name,algorithm,algorithm_id,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit
0,0,1000,A+B,"수학,구현,사칙연산","124,102,121","다국어,디버그,분류",118169,381174,43.753,1,2.29,2 초,128 MB
1,1,1001,A-B,"수학,구현,사칙연산","124,102,121","디버그,분류",97109,159168,72.049,1,1.39,2 초,128 MB
2,2,10998,A×B,"수학,구현,사칙연산","124,102,121","디버그,분류",69367,97906,78.356,1,1.28,1 초,256 MB
3,3,10869,사칙연산,"수학,구현,사칙연산","124,102,121","디버그,분류",68791,150720,52.485,1,1.90,1 초,256 MB
4,4,1008,A/B,"수학,구현,사칙연산","124,102,121","스페셜 저지,분류",67564,248349,33.473,2,2.99,2 초,128 MB


In [13]:
ranker_problems.columns[ranker_problems.isna().any()].tolist()

[]

In [14]:
number2idx = {}
for seq, number in zip(recommend_problems['seq'], recommend_problems['number']) :
    number2idx[number] = seq
len(number2idx)

9267

In [15]:
problems = recommend_problems['number']
len(problems), len(problems.unique())

(9267, 9267)

In [16]:
ranker_problems.info()
ranker_problems.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6322246 entries, 0 to 6322245
Data columns (total 5 columns):
id         int64
user_id    int64
user       object
number     int64
TF         object
dtypes: int64(3), object(2)
memory usage: 241.2+ MB


,id,user_id,user,number,TF
0,0,0,koosaga,1000,o
1,1,0,koosaga,1001,o
2,2,0,koosaga,1002,o
3,3,0,koosaga,1003,o
4,4,0,koosaga,1004,o


In [17]:
# 추천 목록에 있는 문제들만 남긴다
ranker_problems = ranker_problems[ranker_problems['number'].isin(problems)]

In [18]:
ranker_problems.info()
ranker_problems.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6148089 entries, 0 to 6322245
Data columns (total 5 columns):
id         int64
user_id    int64
user       object
number     int64
TF         object
dtypes: int64(3), object(2)
memory usage: 281.4+ MB


,id,user_id,user,number,TF
0,0,0,koosaga,1000,o
1,1,0,koosaga,1001,o
2,2,0,koosaga,1002,o
3,3,0,koosaga,1003,o
4,4,0,koosaga,1004,o


In [19]:
problem_ids = []
for number in ranker_problems['number'] :
    problem_id = number2idx[number]
    problem_ids.append(problem_id)

In [20]:
# 문제 번호 - 문제 seq
ranker_problems['problem_id'] = problem_ids

In [21]:
ranker_problems.info()
ranker_problems.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6148089 entries, 0 to 6322245
Data columns (total 6 columns):
id            int64
user_id       int64
user          object
number        int64
TF            object
problem_id    int64
dtypes: int64(4), object(2)
memory usage: 328.3+ MB


,id,user_id,user,number,TF,problem_id
0,0,0,koosaga,1000,o,0
1,1,0,koosaga,1001,o,1
2,2,0,koosaga,1002,o,45
3,3,0,koosaga,1003,o,2242
4,4,0,koosaga,1004,o,8398


In [22]:
# 맞은 문제, 틀린 문제 가중치 주기
ranker_problems['score'] = ranker_problems['TF'].replace({'o' : 1, 'x': 2})

In [23]:
ranker_problems.info()
ranker_problems.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6148089 entries, 0 to 6322245
Data columns (total 7 columns):
id            int64
user_id       int64
user          object
number        int64
TF            object
problem_id    int64
score         int64
dtypes: int64(5), object(2)
memory usage: 375.2+ MB


,id,user_id,user,number,TF,problem_id,score
0,0,0,koosaga,1000,o,0,1
1,1,0,koosaga,1001,o,1,1
2,2,0,koosaga,1002,o,45,1
3,3,0,koosaga,1003,o,2242,1
4,4,0,koosaga,1004,o,8398,1


In [24]:
# ranker_problems.to_csv("final_data/CSV/ranker_problems.csv", index = False, encoding='utf-8-sig')
# pd.to_pickle(ranker_problems, "final_data/ranker_problems.pkl")

In [26]:
len(ranker_problems['number'].unique())

9267

In [27]:
problems_table = ranker_problems.pivot_table(
        index="number", columns="TF", aggfunc="size", fill_value = 0)

In [28]:
total_cnt = len(ranker_problems['user_id'].unique())
total_cnt

34900

In [29]:
problems_table['cnt'] = problems_table['o'] + problems_table['x']
problems_table['no'] = total_cnt - problems_table['cnt']

In [30]:
problems_table.sort_values(by = 'cnt', ascending=False)[:5000]

TF,o,x,cnt,no
number,,,,
1000,31094,77,31171,3729
2557,30568,67,30635,4265
1001,28746,48,28794,6106
2438,28128,27,28155,6745
2741,27268,93,27361,7539
...,...,...,...,...
2205,34,5,39,34861
1066,28,11,39,34861
17833,39,0,39,34861


### 랭커들의 경험치 계산 및 백준 아이디 경험치 계산
https://ddmanager.tistory.com/122

In [31]:
# 문제 난이도별 경험치, 레벨별 필요 경험치
exps = [320, 480, 672,954,1374,1992,2909,4276,6329,9430,14145,21288,32145,48699,74023,112885,172714,265117,408280,630792,977727,1520366,2371771,3711822,5827560,9178407,14501883,22985485,36546921,58292339,93267742]
problem_exp = {level : exp for level, exp in enumerate(exps)}

levels = [3200, 9590, 13440,19080,27480,39840,72725,106900,158225,235750,353625,638640,964350,1460970,2220690,4545400,5181420,7953510,12248400,18923760,29331810,45610980,71157930,111354660,174826800,367136280,580075320,919419400,1461876840,2331693560]
level_exp = {}

cur_level = 0
for level, exp in enumerate(levels) :
    cur_level += exp
    level_exp[level] = cur_level
    
len(problem_exp), len(level_exp)

(31, 30)

In [32]:
level_df = pd.DataFrame({
    'level' : list(level_exp.keys()),
    'exp' : list(level_exp.values())
})

level_df.info()
level_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
level    30 non-null int64
exp      30 non-null int64
dtypes: int64(2)
memory usage: 608.0 bytes


,level,exp
0,0,3200
1,1,12790
2,2,26230
3,3,45310
4,4,72790


In [33]:
problem_level = recommend_problems[['number', 'level']]
problem_level.info()
problem_level.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9267 entries, 0 to 9266
Data columns (total 2 columns):
number    9267 non-null int64
level     9267 non-null int64
dtypes: int64(2)
memory usage: 144.9 KB


,number,level
0,1000,1
1,1001,1
2,10998,1
3,10869,1
4,1008,2


In [34]:
problem_level['exp'] = problem_level['level'].replace(problem_exp)

# problem_level['exp2'] = list(map(lambda x : problem_exp[x], problem_level['level']))
# print(len(problem_level[problem_level['exp'] != problem_level['exp2']]))

problem_level.info()
problem_level.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9267 entries, 0 to 9266
Data columns (total 3 columns):
number    9267 non-null int64
level     9267 non-null int64
exp       9267 non-null int64
dtypes: int64(3)
memory usage: 217.3 KB


c:\users\multicampus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,number,level,exp
0,1000,1,480
1,1001,1,480
2,10998,1,480
3,10869,1,480
4,1008,2,672


In [35]:
# problem_level.to_csv("final_data/problem_exps.csv", index = False, encoding='utf-8-sig')
# pd.to_pickle(problem_level, "final_data/problem_exps.pkl")

# level_df.to_csv("final_data/level_exps.csv", index = False, encoding='utf-8-sig')
# pd.to_pickle(level_df, "final_data/level_exps.pkl")

In [38]:
def make_problem_exp_dict() :
    problem_exps = pd.read_pickle("final_data/Required_Pickle/problem_exps.pkl")
    problem_exps = dict(zip(problem_exps['number'], problem_exps['exp']))
    
    return problem_exps

def make_level_exp_dict() :
    level_exps = pd.read_pickle("final_data/Required_Pickle/level_exps.pkl")
    level_exps = dict(zip(level_exps['level'], level_exps['exp']))
    
    return level_exps

def get_level(user_id) :
    
    # 0. 문제별 경험치 정보 사전, 레벨 경험치 정보 사전 생성
    problem_exp = make_problem_exp_dict()
    level_exp = make_level_exp_dict()
    
    # 1. 해당 아이디가 맞은 문제들 조회
    problem_numbers = ranker_problems[(ranker_problems["user"] == user_id)]['number']
    
    # 2. 경험치 계산
    exp = sum(list(map(lambda x : problem_exp[x], problem_numbers)))
    
    # 3. 경험치에 맞는 레벨 계산
    level = 0
    for cur_level in range(30, 0, -1) : 
        if level_exp[cur_level - 1] <= exp :
            level = cur_level
            break
            
    return level

In [39]:
get_level('kkwinwin95')

13

### 랭커들의 레벨 계산

In [40]:
ranker_user_id = rankers['user'].unique()
ranker_user_id[:10]

array(['koosaga', 'dotorya', 'xiaowuc1', 'cki86201', 'portableangel',
       'chogahui05', 'jh05013', 'kyo20111', 'jhnah917', 'ainta'],
      dtype=object)

In [41]:
# rankers_with_level.csv에 저장

# import time
# levels = []
# index = 0;
# start = time.time()

# for user_id in ranker_user_id :
#     levels.append(get_level(user_id))
#     index += 1

#     if index % 100 == 0 :
#         print(index, " : ", time.time() - start)
#         start = time.time()

In [43]:
ranker_with_level = pd.read_csv('raw_data/rankers_with_level.csv')

ranker_with_level.info()
ranker_with_level.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34900 entries, 0 to 34899
Data columns (total 5 columns):
id             34900 non-null int64
user           34900 non-null object
correct_cnt    34900 non-null int64
level          34900 non-null int64
calc_level     34900 non-null int64
dtypes: int64(4), object(1)
memory usage: 1.3+ MB


,id,user,correct_cnt,level,calc_level
0,0,koosaga,8842,30,30
1,1,dotorya,7486,27,27
2,2,xiaowuc1,6246,30,27
3,3,cki86201,5900,30,30
4,4,portableangel,5771,24,25


In [44]:
# null값 가지는 행 찾기
ranker_with_level[pd.isnull(ranker_with_level).any(axis=1)]

,id,user,correct_cnt,level,calc_level


In [45]:
ranker_with_level[ranker_with_level['level'].isin([-1, 999])]

,id,user,correct_cnt,level,calc_level
10,10,snrnsidy,3895,999,24
21,21,august14,3236,999,27
29,29,ntopia,2982,999,25
35,35,citizen,2726,999,21
39,39,doju,2643,999,25
...,...,...,...,...,...
34891,34891,cjm2423,57,999,10
34892,34892,ligsun22,57,999,7
34894,34894,seho4815,57,999,6
34896,34896,choopick,57,999,5


In [46]:
new_level = []

ids = []
less_diff = []
user_ids = list(ranker_with_level['user'])
solved_level = list(ranker_with_level['level'])
calc_level = list(ranker_with_level['calc_level'])

for user_id, prev, cur in zip(user_ids, solved_level, calc_level) :    
    if prev in [0, -1, 999] :
        new_level.append(cur)        
    else :
        diff = cur - prev
        
        if diff < -5 or diff > 5 :
            # 계산한 레벨과 차이가 큰 사람들
            ids.append(user_id)
            new_level.append(cur)
        else :
            less_diff.append(user_id)
            new_level.append(prev)

In [47]:
len(ids)

859

In [48]:
ranker_with_level['new_level'] = new_level

ranker_with_level.info()
ranker_with_level.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34900 entries, 0 to 34899
Data columns (total 6 columns):
id             34900 non-null int64
user           34900 non-null object
correct_cnt    34900 non-null int64
level          34900 non-null int64
calc_level     34900 non-null int64
new_level      34900 non-null int64
dtypes: int64(5), object(1)
memory usage: 1.6+ MB


,id,user,correct_cnt,level,calc_level,new_level
0,0,koosaga,8842,30,30,30
1,1,dotorya,7486,27,27,27
2,2,xiaowuc1,6246,30,27,30
3,3,cki86201,5900,30,30,30
4,4,portableangel,5771,24,25,24
5,5,chogahui05,5110,24,25,24
6,6,jh05013,4845,28,27,28
7,7,kyo20111,4134,26,25,26
8,8,jhnah917,4026,30,27,30
9,9,ainta,3989,30,29,30


In [49]:
ranker_with_level.drop(['level', 'calc_level'], axis=1, inplace = True)
ranker_with_level = ranker_with_level.rename({'new_level' : 'level'}, axis = 1)

ranker_with_level.info()
ranker_with_level.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34900 entries, 0 to 34899
Data columns (total 4 columns):
id             34900 non-null int64
user           34900 non-null object
correct_cnt    34900 non-null int64
level          34900 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.1+ MB


,id,user,correct_cnt,level
0,0,koosaga,8842,30
1,1,dotorya,7486,27
2,2,xiaowuc1,6246,30
3,3,cki86201,5900,30
4,4,portableangel,5771,24
5,5,chogahui05,5110,24
6,6,jh05013,4845,28
7,7,kyo20111,4134,26
8,8,jhnah917,4026,30
9,9,ainta,3989,30


In [50]:
# ranker_with_level.to_csv("final_data/rankers.csv", index = False, encoding='utf-8-sig')
# pd.to_pickle(ranker_with_level, "final_data/rankers.pkl")